In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt

In [2]:
#Read in J1939Faults. low_memory = false because of warning about mixed type columns.
faults = pd.read_csv('../data/J1939Faults.csv',
                     index_col = 'RecordID',
                     parse_dates = ['EventTimeStamp', 'LocationTimeStamp'],
                     low_memory = False)
#remove faults that occur at service locations. Had to remove the last digit because the rows dont have it.
service_location1 = faults[(faults["Latitude"] == 36.066666) & (faults["Longitude"] == -86.434722)]

service_location2 = faults[(faults["Latitude"] == 35.588333) & (faults["Longitude"] == -86.443888)]

service_location3 = faults[(faults["Latitude"] == 36.1950) & (faults["Longitude"] ==  -83.174722)]

#concat service_locations for removal
service_locations = pd.concat([service_location1, service_location2, service_location3])

#drop service locations
faults_cleaned = faults.drop(service_locations.index)

#remove EquipmentIDs that are longer than 5 Characters per the README
faults_cleaned = faults_cleaned[faults_cleaned['EquipmentID'].map(len) <= 5]

#removed columns that only contained null values or would not be valuable because they're unique to the truck.
faults_cleaned = faults_cleaned.drop(['actionDescription', 'faultValue', 'ecuSerialNumber',
                                      'ecuSource', 'MCTNumber'], axis = 1).reset_index()

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#Limit to years 2015-2020, Had to add down to seconds because it kept changing the datatype after filtering
faults_cleaned = faults_cleaned[(faults_cleaned['EventTimeStamp'] >= '2015-01-01 00:00:01') & (faults_cleaned['EventTimeStamp'] <= '2020-12-31 12:59:59')]



In [4]:
#prepare derates for dummization
faults_cleaned.loc[faults_cleaned['spn'] != 1569, 'spn_derate'] = 'neither'
faults_cleaned.loc[faults_cleaned['spn'] != 5246, 'spn_derate'] = 'neither'
faults_cleaned.loc[faults_cleaned['spn'] == 5246, 'spn_derate'] = 'full'
faults_cleaned.loc[faults_cleaned['spn'] == 1569, 'spn_derate'] = 'partial'  


faults_cleaned = faults_cleaned.set_index('RecordID')

#get dummies
derates = pd.get_dummies(faults_cleaned['spn_derate'])
derates

#merge back with original dataset
faults_cleaned = faults_cleaned.merge(derates, left_on = 'RecordID', right_on = 'RecordID') 

In [5]:
#sorts and groups by truck. then sorts by event timestamp. this made the diff() calculate correctly 
#    ¯\_(ツ)_/¯
#code shamelessly stolen from https://arccoder.medium.com/pandas-sort-within-groups-e1f3b6a10a3f
faults_cleaned = faults_cleaned.sort_values(['EquipmentID'], ascending=True) \
    .groupby(['EquipmentID'], sort=False) \
    .apply(lambda x: x.sort_values(['EventTimeStamp'], ascending=True)) \
    .reset_index(drop=True)


faults_cleaned['timedelta'] = faults_cleaned.groupby('EquipmentID')['EventTimeStamp'].diff()

In [54]:
faults['EventTimeStamp'].dt.to_period('Y').value_counts()

2016    332383
2015    325779
2017    254851
2018    144510
2019    112075
2020     17188
2011       244
2000       219
2010        58
2009        24
2026         3
2002         1
Freq: A-DEC, Name: EventTimeStamp, dtype: int64

In [6]:
faults_cleaned['EventTimeStamp'].dt.to_period('Y').value_counts()

2016    332175
2015    325536
2017    254680
2018    143289
2019    111321
2020     17179
Freq: A-DEC, Name: EventTimeStamp, dtype: int64

### Based on the distribution of years I think it would be best to limit the dataset to years 2015-2020 

#### 2015-2020 are consecutive while before 2015 it skips years 2014, 2013, 2012 and picks back up at 2011. It's possible that some trucks from 2014 carry over into 2015, but unlikely any from 2011 are carried over since they limit their lease to 4 years.

# In the full dataset
### How many trucks have a full derate/partial derate? 
### out of 1045 total trucks in the dataset 

Interestingly, when filtering down for years 2015-2020, Full derates went down from 211 to 210 but the partial remained unchanged. 

#### 526 have either a partial OR a full (we have some that have a partial but not a full and some that have a full but not a partial)

#### 210 have a full derate

#### 498 have a partial

#### 182 of the trucks that have a partial AND have a full derate.

#### 519 have neither a partial NOR a full

In [116]:
len(faults_cleaned['EquipmentID'].unique())

1045

In [7]:
full = faults_cleaned[(faults_cleaned['spn'] == 5246)]
full = full['EquipmentID'].unique()

len(full)

210

In [8]:
partial = faults_cleaned[(faults_cleaned['spn'] == 1569)]
partial = partial['EquipmentID'].unique()

len(partial)

498

In [9]:
intersection = np.intersect1d(full, partial)

len(intersection)

182

In [120]:
both = faults_cleaned[((faults_cleaned['spn']== 5246) | (faults_cleaned['spn']==1569))]

both = both['EquipmentID'].unique()

len(both)

526

In [121]:
no_derates = faults_cleaned[~faults_cleaned['EquipmentID'].isin(both)]
#no_derates = no_derates['EquipmentID'].unique()
len(no_derates)

235153

In [ ]:
faults_cleaned['timedelta'].describe()

In [41]:
from tqdm.notebook import tqdm

In [42]:
#Create list for trucks that have a full derate
full = faults_cleaned[(faults_cleaned['spn'] == 5246)]
full = full['EquipmentID'].unique()

#create an empty list to hold the derated_trucks
derated_trucks = []
#iterate through the dataframe and return the trucks that derate and the preceeding 10 rows. 
for truck_num in tqdm(full):
    truck = faults_cleaned[faults_cleaned['EquipmentID']==truck_num]
    first_full_index = truck[truck['spn']==5246].index[0]
    derated_trucks.append(truck.loc[first_full_index-10:first_full_index])
#concat the list into a new dataframe   
derated_trucks = pd.concat(derated_trucks)


  0%|          | 0/210 [00:00<?, ?it/s]

In [43]:
derated_trucks

,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,spn_derate,full,neither,partial,timedelta
99,1070647,2015-02-25 13:53:08,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,1569,31,True,1,1329,39.399583,-82.974768,2015-02-25 13:56:31,partial,0,0,1,NaT
100,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,unknown,5246,0,True,1,1329,39.399583,-82.974768,2015-02-25 13:56:31,full,1,0,0,0 days 00:00:00
2039,2865286,2015-06-09 15:40:51,NaN,unknown,unknown,unknown,50353,0,True,2,1339,35.749027,-78.868333,2015-06-09 15:41:28,neither,0,1,0,0 days 05:41:26
2040,2865323,2015-06-09 15:43:36,NaN,unknown,unknown,unknown,50353,0,False,2,1339,35.745833,-78.905185,2015-06-09 15:43:32,neither,0,1,0,0 days 00:02:45
2041,2881610,2015-06-10 11:42:08,NaN,unknown,unknown,unknown,5394,7,True,1,1339,35.756712,-77.869444,2015-06-10 11:42:43,neither,0,1,0,0 days 19:58:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174613,60855543,2018-09-06 15:38:45,Incorrect Data J1939 Network #1 Primary Vehicl...,BB41103* BB41104*,EC60-adv,BNDWS,639,2,False,127,306,36.066435,-86.433518,2018-09-06 15:38:40,neither,0,1,0,0 days 00:25:40
1174614,60904453,2018-09-07 05:45:39,Data Drifted High Aftertreatment 1 Outlet NOx,05317106*04075152*092613211021*09300006*G1*BDR*,6X1u13D1500000000,CMMNS,3226,20,True,1,306,36.167777,-86.529120,2018-09-07 05:46:16,neither,0,1,0,0 days 14:06:54
1174615,60904613,2018-09-07 05:48:43,Condition Exists NOx limits exceeded due to In...,05317106*04075152*092613211021*09300006*G1*BDR*,6X1u13D1500000000,CMMNS,4094,31,True,1,306,36.169907,-86.587731,2018-09-07 05:49:20,neither,0,1,0,0 days 00:03:04
1174616,60909415,2018-09-07 06:47:05,Condition Exists Engine Protection Torque Derate,05317106*04075152*092613211021*09300006*G1*BDR*,6X1u13D1500000000,CMMNS,1569,31,True,1,306,36.173240,-86.776990,2018-09-07 06:47:46,partial,0,0,1,0 days 00:58:22


In [44]:
T_2_towing = []
for truck_num in tqdm(full):
    truck = faults_cleaned[faults_cleaned['EquipmentID']==truck_num]
    first_full_index = truck[truck['spn']==5246].index[0]
    T_2_towing.append(truck.loc[first_full_index-1:first_full_index])
    
T_2_towing = pd.concat(T_2_towing)

  0%|          | 0/210 [00:00<?, ?it/s]

In [91]:
derated_trucks['spn'].value_counts()

1569      376
111       334
5246      210
1761      172
3362       69
         ... 
4765        1
520953      1
4376        1
4382        1
1028        1
Name: spn, Length: 133, dtype: int64

In [65]:
T_2_towing['timedelta'].describe()

count                          414
mean     1 days 06:00:33.492753623
std      7 days 17:23:31.865378688
min                0 days 00:00:00
25%         0 days 00:12:36.750000
50%         0 days 01:30:22.500000
75%         0 days 05:56:43.500000
max              129 days 12:48:45
Name: timedelta, dtype: object

In [64]:
T_2_towing['spn'].value_counts()

5246    210
1569    104
111      17
1761     10
3362     10
3226      9
5394      6
4342      4
639       3
4340      3
3360      3
3216      3
4094      3
3031      3
641       3
3364      2
6802      2
96        2
74        2
1068      2
790       1
5024      1
929       1
3361      1
1483      1
791       1
4375      1
789       1
4364      1
4334      1
3610      1
153       1
649       1
5742      1
612       1
596       1
70        1
5848      1
Name: spn, dtype: int64

In [69]:
faults_cleaned.head()

,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,spn_derate,full,neither,partial,timedelta
0,1059893,2015-02-25 06:17:50,Low (Severity Medium) Battery Potential / Powe...,04993120*00001782*082113134117*07700053*I0*BBZ*,6X1u10D1500000000,CMMNS,444,18,True,1,1327,36.066805,-86.433981,2015-02-25 06:22:31,neither,0,1,0,NaT
1,1059892,2015-02-25 06:17:50,Low (Severity Low) Engine Coolant Level,04993120*00001782*082113134117*07700053*I0*BBZ*,6X1u10D1500000000,CMMNS,111,17,True,1,1327,36.066805,-86.433981,2015-02-25 06:22:30,neither,0,1,0,0 days 00:00:00
2,1061595,2015-02-25 07:40:59,Low (Severity Low) Engine Coolant Level,04993120*00001782*082113134117*07700053*I0*BBZ*,6X1u10D1500000000,CMMNS,111,17,False,1,1327,36.067083,-86.434722,2015-02-25 06:24:30,neither,0,1,0,0 days 01:23:09
3,1062652,2015-02-25 08:24:49,Low (Severity Medium) Battery Potential / Powe...,04993120*00001782*082113134117*07700053*I0*BBZ*,6X1u10D1500000000,CMMNS,444,18,False,1,1327,36.067083,-86.434722,2015-02-25 06:24:30,neither,0,1,0,0 days 00:43:50
4,2022694,2015-04-22 09:10:12,Low Voltage (Aftertreatment 1 Particulate Trap...,04993120*00001782*082113134117*07700053*I0*BBZ*,6X1u10D1500000000,CMMNS,3251,4,True,1,1327,36.194861,-83.174768,2015-04-22 09:10:48,neither,0,1,0,56 days 00:45:23


In [82]:
#bring in diagnostics and pivot
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv') \
    .pivot(index = 'FaultId', columns = 'Name', values = 'Value')

faults_all = faults_cleaned.merge(diagnostics, left_index = True, right_on = 'FaultId')

faults_all['DistanceLtd'] = faults_all['DistanceLtd'].str.replace(',','.').astype('float64')

In [130]:
derated_trucks = derated_trucks.merge(diagnostics, left_index = True, right_on = 'FaultId')
derated_trucks['DistanceLtd'] = derated_trucks['DistanceLtd'].str.replace(',','.').astype('float64')

In [126]:
#Merge the trucks that don't derate with the diagnostics data.

no_derates = no_derates.merge(diagnostics, left_index = True, right_on = 'FaultId')

no_derates['DistanceLtd'] = no_derates['DistanceLtd'].str.replace(',','.').astype('float64')

In [83]:
faults_all['DistanceLtd'].describe()

count    558274.000000
mean     361953.520802
std      142541.624015
min           0.000000
25%      267989.000000
50%      373051.200000
75%      479221.400000
max      657065.000000
Name: DistanceLtd, dtype: float64

In [131]:
derated_trucks['DistanceLtd'].describe()

count      1041.000000
mean     353430.351207
std      144572.481053
min        2166.970000
25%      253580.400000
50%      339249.000000
75%      484170.200000
max      613457.300000
Name: DistanceLtd, dtype: float64

In [129]:
no_derates['DistanceLtd'].describe()

count    109177.000000
mean     355629.163211
std      141233.229357
min           0.000000
25%      261105.500000
50%      344424.000000
75%      482705.900000
max      657065.000000
Name: DistanceLtd, dtype: float64

In [86]:
faults_all[faults_all['active'] == 0].count()

ESS_Id                       551222
EventTimeStamp               551222
eventDescription             525645
ecuSoftwareVersion           413069
ecuModel                     520214
ecuMake                      520214
spn                          551222
fmi                          551222
active                       551222
activeTransitionCount        551222
EquipmentID                  551222
Latitude                     551222
Longitude                    551222
LocationTimeStamp            551222
spn_derate                   551222
full                         551222
neither                      551222
partial                      551222
timedelta                    551141
AcceleratorPedal             246533
BarometricPressure           272381
CruiseControlActive          266980
CruiseControlSetSpeed        267700
DistanceLtd                  272341
EngineCoolantTemperature     272414
EngineLoad                   272169
EngineOilPressure            272475
EngineOilTemperature        

In [90]:
faults_all.groupby('EquipmentID')['DistanceLtd'].apply(lambda x: x.isna().sum()/len(x)).describe()

count    1001.000000
mean        0.508736
std         0.078617
min         0.000000
25%         0.493252
50%         0.502488
75%         0.525974
max         1.000000
Name: DistanceLtd, dtype: float64

In [128]:
no_derates.groupby('EquipmentID')['DistanceLtd'].apply(lambda x: x.isna().sum()/len(x)).describe()

count    487.000000
mean       0.511389
std        0.092924
min        0.000000
25%        0.485937
50%        0.501969
75%        0.541667
max        1.000000
Name: DistanceLtd, dtype: float64

If you change the lease length. What if renewing at 3 years is better than 4 years? At what point in their life do they start seeing derates. 
How often are fault codes thrown at different mileages. 
Full derates as mileage changes. 

There is a difference between trucks that don't experience derates and ones that do with the codes leading up to the derate.

3251 in non-derates. 
